# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

experiment='cxilu9218'
# experiment='cxix38318'
pullDataFromUser='igabalsk' # update me to mrware or igablsk
RESULTSPATH=('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,pullDataFromUser)).strip()
# Load in the pythonBatchMagic |library
from scipy.ndimage import gaussian_filter1d
import numpy as np
import matplotlib.pyplot as plt

# Load point data from pkl (preferred)

In [ ]:
import os
import pickle
def load_obj(filename ):
    """
    Loads object from name.pkl and returns its value

    Args:
        filename: String designating directory and name of file, ie. /Folder/Filename, where Filename.pkl is the object

    Returns:
        The value of the object in filename
    """
    try:
        with open(filename + '.pkl', 'rb') as f:
            print filename+" remembered!"
            return pickle.load(f)
    except IOError as e:
        print "IOError: Did you load the correct file? %s" % filename
        raise e
def save_obj(obj, filename ):
    """
    Saves object from filename.pkl

    Args:
        obj: The python object to save
        filename: String designating directory and name of file, ie. /Folder/Filename, where Filename.pkl is the object
    """
    with open(filename + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

exclude_keys = ['Qs','phis','bin_sizes']

def combineRuns(runNumbers, path=RESULTSPATH, prefix='all'):
    detArrays = {}
    for idx,run in enumerate(runNumbers):
        if idx == 0:
            detArrays = load_obj(path+'/%sData-run-%d' % (prefix,run))
        else:
            try:
                detArrays0 = load_obj(path+'/%sData-run-%d' % (prefix,run))
                for key in detArrays.keys():
                    if key in exclude_keys:
                        continue
                    try:
                        detArrays[key] = np.append( detArrays[key], detArrays0[key], axis=0 )
                    except KeyError as ke:
                        print('Dropping key %s since it is not in %d' % (key,run))
                        detArrays.pop(key, None)
            except IOError as ioe:
                print(str(ioe))
                continue
    return detArrays

def downsampleRun(runNumber, path=RESULTSPATH):
    detArrays = load_obj(path+'/allData-run-%d' % runNumber)
    bin_sizes = detArrays['bin_sizes']
    qphirois = detArrays['qphirois']
    qphirois[np.isnan(qphirois)] = 0
    qbin_weights = np.repeat(np.expand_dims(bin_sizes,axis=0),len(qphirois),axis=0)
    rois = np.average(qphirois,axis=-1,weights=qbin_weights)
    allIdx = np.ones_like(detArrays['seconds']).astype(bool)
    newdetArrays = {}
    exclude_keys = ['qphirois','Qs','phis','bin_sizes']
    for key in detArrays.keys():
        if key in exclude_keys:
            continue
        try:
            newdetArrays[key]= np.copy(detArrays[key][allIdx,:])
        except IndexError as ie:
            newdetArrays[key]= np.copy(detArrays[key][allIdx] )
    newdetArrays['rois'] = np.copy(rois[allIdx,:])
    newdetArrays['Qs'] = np.copy(detArrays['Qs'])
    newdetArrays['bin_sizes'] = np.sum(detArrays['bin_sizes'],axis=-1)
        
    save_obj( newdetArrays, path +'/filteredData-run-%d' % runNumber )
    np.savez(path +'/filteredData-run-%d' % runNumber, **newdetArrays)
    import h5py
    with h5py.File(path +'/filteredData-run-%d.h5' % runNumber,'w') as f:
        for key in newdetArrays.keys():
            f.create_dataset(key,data=np.array(newdetArrays[key]).astype(float))


In [ ]:
runNumbers = [45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,]#61,62,63,64,]#65,67,68,69,70,71] # day 1
# runNumbers = [91,92,93,94,95,96,97,98,99,100,] # day 2
# runNumbers = [121,122,123,124,125,126,127,128,129,130,131]
runNumbersRange = '[%d - %d]' % (min(runNumbers),max(runNumbers))

detArrays = combineRuns(runNumbers,prefix='filtered')

In [ ]:
p = np.array([ 2.94720875e-06, -1.36213787e-03]) # LV11
p = np.array([2.95684259e-06, -1.43969413e-03])  # LU92
p = np.array([-9.36209506e-10,  3.76314033e-06, -1.63476074e-03]) # LU92 quadratic fit
pos0 = 80.554 # LV11
pos0 = 56.35 # LU92
stage_fs = -2*(detArrays['stageencoder']-pos0) / (3e-4)

# Radiial rois vs. pixel and vs. Q

In [ ]:
Q = detArrays['Qs']
print detArrays['rois'].shape
plt.figure()
meanSig = np.nanmean(detArrays['rois'][1:,],0)
plt.plot(Q,Q*meanSig)
plt.xlabel('Q')
plt.ylabel('Q * I(Q)')
plt.show()
goodQIdx = np.concatenate((np.array([1.0]),np.diff(meanSig)<0.0001))
goodQIdx[goodQIdx==0]=np.nan
filteredrois = goodQIdx*detArrays['rois']
meanSig = np.nanmean(filteredrois,0)
plt.plot(Q,Q*meanSig)
plt.xlabel('Q')
plt.ylabel('Q * I(Q)')
plt.show()

### Apply detector response function to the average scattering pattern and compare to the theoretical signal

In [ ]:
def normalizedPlot(x,y,s=1,**kwargs):
    plt.semilogy(x, y/np.nanmean(y[(x>1)&(x<2.5)])*s,**kwargs)

In [ ]:
import h5py
def loadfile(filename):
    mol = {}
    with h5py.File(filename,'r') as f:
        for name, data in f.items():
            mol[name]=f[name][()]
            print name
    return mol

molecule = 'CS2'
mol = loadfile('/cds/home/i/igabalsk/xray/diffraction_simulation/isotropic_scattering_%s.h5' % molecule)

interpolatedMol = np.interp(Q, mol['QQ_1d'], mol['isotropic_scattering_1d'])

normalizedPlot(Q,meanSig,s=1)
normalizedPlot(Q,interpolatedMol)
# plt.savefig('masked_mean_withresponse.png')

# From seconds, nanoseconds, and fiducials generate the labtime

In [ ]:
import datetime
import matplotlib.dates as dates
def getLabtime(second, microseconds):
    refdate = datetime.datetime(1970,1,1)
    td = datetime.timedelta(seconds=second,microseconds=microseconds)
    return refdate+td
labtime = np.array([getLabtime( s, float(ns)/1000. ) for s,ns in zip(detArrays['seconds'],detArrays['nanoseconds'])])

# Outlier rejection and time binning

In [ ]:
goodIdx = ( detArrays['xrayEnergy']>.2 ) 
goodIdx = goodIdx & (np.nansum(detArrays['rois'],-1) > 0.01) 
goodIdx = goodIdx & (~np.isnan(detArrays['uvint1'])) 
goodIdx = goodIdx & ( detArrays['xrayOn'].astype(bool))
offIdx = goodIdx & (~detArrays['laserOn'].astype(bool))
goodIdx = goodIdx & ( detArrays['laserOn'].astype(bool))

dp=75
goodIdx = goodIdx &( (detArrays['ttfltpos']>500-dp)  & (detArrays['ttfltpos']<500+dp)  )
goodIdx = goodIdx &( (detArrays['ttfltposfwhm']>5) )
goodIdx = goodIdx &( (detArrays['ttfltposampl']>.005) )

pos = detArrays['stageencoder'][goodIdx] 
ttpos = detArrays['ttfltpos'][goodIdx]
goodRois = detArrays['rois'][goodIdx,:]
offRois =  detArrays['rois'][offIdx,:]
print 'Good shots:',np.sum(goodIdx>0)
print 'Total shots:',detArrays['ttfltpos'].size

ttpoly = [2.95684259e-06, -1.43969413e-03] # LV11
ttpoly = [2.95684259e-06, -1.43969413e-03] # LU92
ttpoly = [-9.36209506e-10,  3.76314033e-06, -1.63476074e-03] # LU92 quadratic fit
def ttcorr(ttpos,ttpoly):
    return ttpoly[0]*ttpos**2+ttpoly[1]*ttpos+ttpoly[2]
truepos = -2*(pos-pos0) / (3e-4)  - ttcorr(ttpos,ttpoly)*1.0e6
posfs = -2*(pos-pos0) / (3e-4) 

roio = np.nansum(offRois,-1)

# Preprocess the radial rois by normalizing by xray energy and subtracting the nearest off shot

In [ ]:
def makeWeights( rois, goodIdx, offIdx, labtime, subNearest=0 ):
    offtime = labtime[offIdx]
    goodtime = labtime[goodIdx]
    offrois = rois[offIdx,:]
    goodrois = rois[goodIdx,:]
    
    osum = np.nansum(offrois,-1)
    offroiN = ((offrois.T)/(osum.T)).T
    gsum = np.nansum(goodrois,-1)
    groiN = ((goodrois.T)/(gsum.T)).T
    
    offDelta = offtime-datetime.datetime(1970,1,1)
    offDelta_s = np.array([dt.total_seconds() for dt in offDelta])
    offArg = np.argsort( offDelta_s )
    offMin = np.min(offDelta_s)
    offMax = np.max(offDelta_s)
    offRange = offMax-offMin
    noff = offtime.size
    
    offroiNS = offroiN[offArg,:]
    
    weights = np.zeros_like(goodrois)
    for idx,atime in enumerate(goodtime):
#         print(idx,goodtime.size)
        dtime =( atime-datetime.datetime(1970,1,1)).total_seconds()
        oidx = int((dtime - offMin)/offRange*noff)
        if oidx < 0:
            oidx = 0
        if oidx >= noff:
            oidx = noff-1
        weights[idx,:] = groiN[idx,:] - subNearest*offroiNS[oidx,:]
        
    return weights

In [ ]:
weightMe = makeWeights( detArrays['rois'], goodIdx, offIdx, labtime, subNearest=0 )

# Rebin the shot-by-shot data into time bins (without timetool correction)

In [ ]:
db = 1e-2
bins = np.unique(pos) - db
bins = np.append(bins, bins[-1]+ 2*db)
nb = bins.size
nr = goodRois.shape[1]

def createBinsFromCenters(centers):
    bins = []
    nc = centers.size
    for idx,c in enumerate(centers):
        if idx == 0:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c-dc)
            bins.append(c+dc)
        elif idx == nc-1:
            dc = np.abs( c - centers[idx-1])/2.
            bins.append(c+dc)
        else:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c+dc)
#         print(dc)
    return np.array(bins)

def determineGoodCenters( pos ):
    upos = np.unique(pos)
    gpos = []
    for idx,up in enumerate(upos):
        c = np.sum(upos == up)
        if c>10:
            gpos.append()
        
bins = createBinsFromCenters(  np.sort(np.unique( np.round(pos,decimals=2),axis=None)).flatten())
centers = np.sort(np.unique(np.round(pos,decimals=2),axis=None)).flatten()
centersfs = 2*np.flip(np.array(centers)-pos0) / (3e-4)
# bins = np.append(bins, 80.73)
binspos = bins
# print(bins)
nb = bins.size
# weightMe = ((goodRois.T)/(roi1.T)).T
# weightMe = (goodRois)
print(pos.size)
print(weightMe.shape)

radialHist = np.zeros((nb-1,nr))
radialAvg = np.zeros((nb-1,nr))

counts,edges = np.histogram( pos,bins=bins)
for ir in range(nr):

    radialHist[:,ir],edges = np.histogram( pos,bins=bins, weights=weightMe[:,ir])
    radialAvg[:,ir] = radialHist[:,ir] / counts
    

# Plot the timebin centers and the number of events in each bin

In [ ]:
centers = np.sort(np.unique(np.round(pos,decimals=2),axis=None)).flatten()
centerspos = centers
plt.figure()
# plt.plot(-2*(centers-t0_nominal) / (3e-4), counts)
plt.plot(centers, counts,'.-')
plt.xlabel('delay pos')
plt.ylabel('counts')
plt.show()

# Update cutoff to reflect bad points above

In [ ]:
cutoff = 1000

In [ ]:
plot2d= (radialAvg)[counts>cutoff,:] #/avgAll

rcent = centers[counts>cutoff]
rcentfs = -2*(rcent-pos0) / (3e-4)
# subAll = np.mean( plot2d[:,:], 0 ) # subtract first 3 delays
# avgAll = np.mean( plot2d[-2:,:][:,:], 0 ) # subtract first 3 delays
# subAll = np.mean( plot2d[:,:], 0 )
subAll = np.mean(((offRois.T)/(roio.T)).T,0) # goose subtraction
avgAll = np.mean(((offRois.T)/(roio.T)).T,0)
# plot2d = (plot2d-subAll) /avgAll

gf = lambda x: gaussian_filter1d(x,2,axis=-1)
plot2d = (gf(plot2d)-gf(subAll)) / gf(avgAll)
plot2d = gaussian_filter1d(plot2d,1,axis=0)
# plot2d = (plot2d-subAll) / avgAll

dv = 4e-3
# dv = 1e-5
# dv = .001
# qs = x/ 31578.94736842 * 3.5
print Q.shape, rcent.shape, plot2d.shape
plt.pcolormesh(Q, rcentfs , plot2d, vmin = -dv, vmax = dv )


# plt.pcolormesh(Q, -2*(rcent-pos0) / (3e-4) , plot2d)#, vmin = -dv, vmax = dv )

# plt.pcolormesh(Q, rcent, plot2d)#, vmin = -dv, vmax = dv )
plt.colorbar()

plt.xlabel('Q (iA)')
plt.ylabel('delay (fs)')
plt.title('(I - I(off) )/I(off)')
# plt.savefig('/cds/home/m/mrware/Documents/cxix38318-run37.png')

In [ ]:
fig = plt.figure(figsize=(9,6))
runNumbersRange = '[%d - %d]' % (min(runNumbers),max(runNumbers))
for idx,delay in enumerate(rcentfs):
    plt.plot(Q, plot2d[idx,:], label='%.2f ps'% (delay/1000),linewidth=2 )
plt.ylim([-.015,.015])
plt.xlabel('Q',fontsize=20)
plt.ylabel('dI/I',fontsize=20)
plt.title('Day 1: Runs %s'%runNumbersRange,fontsize=20)
ax = plt.gca()
ax.tick_params(axis='both',length=5,width=2,labelsize=15)
# ax = fig.add_axes([1,0, 0.6, 0.75])
plt.legend(fontsize=15,bbox_to_anchor=(1, 1), loc='upper left',ncol=2)
# plt.savefig('day 1 lineout signal.png')
plt.show()

# Repeat with timetool correction

Specify the centers you'd like to use for timetool bining

Histogram shows the number of counts in each bin you'll generate

In [ ]:
# ttpoly = [2.95684259e-06, -1.43969413e-03]
ttpoly = np.array([ 2.94720875e-06, -1.36213787e-03])
ttpoly = [-9.36209506e-10,  3.76314033e-06, -1.63476074e-03] # LU92 quadratic fit
def ttcorr(ttpos,ttpoly):
    return ttpoly[0]*ttpos**2+ttpoly[1]*ttpos+ttpoly[2]

truepos = -2*(pos-pos0) / (3e-4)  + ttcorr(ttpos,ttpoly)*1.0e6 # correct
# truepos = -2*(pos-pos0) / (3e-4)  - ttcorr(ttpos,ttpoly)*1.0e6 # wrong
posfs = -2*(pos-pos0) / (3e-4) 

plt.figure(figsize=(9,6))
plt.hist(truepos,bins=1000)
ax = plt.gca()
ax.tick_params(axis='both',length=5,width=2,labelsize=15)
plt.xlabel('pump-probe delay (fs)',fontsize=20)
plt.ylabel('frames in timebin',fontsize=20)
# plt.xlim([-10000,10000])
# plt.xlim([-1400,1200])
plt.title('pump-probe delay histogram')

plt.title('Runs %s'%runNumbersRange,fontsize=20)
# usecenters = np.array([3000,2000])
usecenters =  np.arange(-1500,500,15)
# usecenters = np.concatenate((np.array([-2000,-1000]),usecenters,np.array([2000,3000,4000,5000])))
# usecenters = np.append(usecenters,np.array([-2500]))

In [ ]:
db = 1e-3
bins = np.unique(pos) - db
bins = np.append(bins, bins[-1]+ 2*db)
nb = bins.size
nr = goodRois.shape[1]

def createBinsFromCenters(centers):
    bins = []
    nc = centers.size
    for idx,c in enumerate(centers):
        if idx == 0:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c-dc)
            bins.append(c+dc)
        elif idx == nc-1:
            dc = np.abs( c - centers[idx-1])/2.
            bins.append(c+dc)
        else:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c+dc)
#         print(dc)
    return np.array(bins)

def determineGoodCenters( pos ):
    upos = np.unique(pos)
    gpos = []
    for idx,up in enumerate(upos):
        c = np.sum(upos == up)
        if c>10:
            gpos.append()
        
# bins = np.flip(-2*(createBinsFromCenters(  np.round(usecenters,decimals=2) )-56.35)/(3.0e-4))
# centersfs = np.flip(-2*(np.round(usecenters,decimals=2)-56.35)/(3.0e-4))

bins = createBinsFromCenters( np.array(usecenters) )

centersfs = usecenters

#### use rough times
# bins = np.flip(-2*(binspos-56.35)/(3.0e-4))
# centersfs = -2*(centerspos-56.35)/(3.0e-4)

# print(bins)
# -(pos-56.35) / (3e-4) 
nb=bins.size
# weightMe = ((goodRois.T)/(roi1.T)).T
print(weightMe.shape, truepos.shape)

radialHist = np.zeros((nb-1,nr))
radialAvg = np.zeros((nb-1,nr))
counts,edges = np.histogram( truepos,bins=bins)
for ir in range(nr):

    radialHist[:,ir],edges = np.histogram( truepos,bins=bins, weights=weightMe[:,ir])
    radialAvg[:,ir] = radialHist[:,ir] / counts


In [ ]:
cutoff = 300
plt.figure(figsize=(9,6))
plt.plot(centersfs, counts,'.-',linewidth=2,markersize=10)
# print(centersfs).-
plt.ylim([cutoff,np.max(counts)+500])
plt.xlabel('binned delay (fs)',fontsize=20)
plt.ylabel('counts in bin',fontsize=20)
plt.title('Runs %s'%runNumbersRange,fontsize=20)
ax = plt.gca()
ax.tick_params(axis='both',length=5,width=2,labelsize=15)
plt.show()

In [ ]:
plot2d= (radialAvg)[counts>cutoff,:] #/avgAll
#normalize each time bin

rcent=centersfs[counts>cutoff]
subAll = np.mean( plot2d[:5,:], 0 )
# subAll = np.mean( plot2d[:,:], 0 )
# avgAll = np.mean( plot2d[:,:], 0 )
avgAll = np.mean(((offRois.T)/(roio.T)).T,0)
subAll = np.mean(((offRois.T)/(roio.T)).T,0)
# plot2d = (plot2d-subAll) #/avgAll
# dv = .0001
# plot2d = (plot2d-subAll) /avgAll

gf_q = lambda x: gaussian_filter1d(x,3,axis=-1)
gf_t = lambda x: gaussian_filter1d(x,0.7,axis=0)
plot2d = (gf_q(plot2d)-gf_q(subAll)) /gf_q(avgAll)
plot2d = gf_t(plot2d)
# plot2d = gaussian_filter1d(plot2d,1,axis=0)

# dv = 1e
dv = .005
# dv = .01
# qs = x/ 31578.94736842 * 3.5
plt.figure(figsize=(10,8))
plt.pcolormesh(Q, rcent, plot2d, vmin = -dv, vmax = dv )
cbar = plt.colorbar()
cbar.ax.tick_params(length=5,width=2,labelsize=15)
plt.xlabel('Q (iA)',fontsize=20)
plt.ylabel('delay (fs)',fontsize=20)
plt.title('(I - I(off))/I(off) (with timetool)',fontsize=20)
ax = plt.gca()
ax.tick_params(axis='both',length=5,width=2,labelsize=15)
# plt.savefig('day 1 colorplot signal.png')
plt.xlim([0.5,4])
plt.show()

# Temporal fourier analysis

In [ ]:
ridx=(rcent>-700)&(rcent<=1100)

for2dfft =plot2d[ridx,:]
for2dfft = for2dfft - np.mean(for2dfft,0)
fftall = np.fft.fftshift(np.fft.fft(for2dfft , axis=0 ),axes=(0,))
ws = np.fft.fftshift(np.fft.fftfreq(n=for2dfft.shape[0],d=(rcent[1]-rcent[0])/1000.))*2*np.pi
dw = ws[2]-ws[1]

blur_f = 2
gf_f = lambda x: gaussian_filter1d(np.real(x),blur_f,axis=0)+1.0j*gaussian_filter1d(np.imag(x),blur_f,axis=0)
fftfilt = gf_f(fftall)
QQ,WW = np.meshgrid(Q,ws)
plt.pcolormesh(Q,ws,np.abs(WW)*np.abs(fftfilt)**2)
plt.colorbar()
plt.clim([0,3e-2])
plt.xlim([0.5,4])
plt.xlabel('Q (iA)')
plt.ylabel('w (rad-THz)')
plt.title('FT(dI/I) for runs '+str(runNumbersRange))
plt.show()
print fftfilt.shape

In [ ]:
from scipy.signal import lombscargle
def get_lombscargle(Qs, ts, signal):
    omegas = np.fft.fftshift(np.fft.fftfreq(n=ts.size,d=np.mean(np.diff(ts))) )*2*np.pi
    goodOmegas = omegas[omegas>0]
    goodOmegas = np.linspace(np.min(goodOmegas),1*np.max(goodOmegas),len(goodOmegas))
    output = np.zeros((2*goodOmegas.size,Qs.size))
    nw = len(goodOmegas)
    for qIdx, q in enumerate(Qs):
        output[nw:,qIdx] = lombscargle(ts,signal[:,qIdx],goodOmegas,precenter=True)
        output[:nw,qIdx] = np.flip(output[nw:,qIdx],axis=0)
    return 1000*np.concatenate((-1*np.flip(goodOmegas),goodOmegas)), output

ridx=(rcent>-600)&(rcent<=400)
for2dfft =plot2d[ridx,:]
for2dfft = for2dfft - np.mean(for2dfft,0)
fftall = np.fft.fftshift(np.fft.fft(for2dfft , axis=0 ),axes=(0,))
omegas_ls, ls = get_lombscargle(Q,rcent[ridx],plot2d[ridx,:])

# omegas_ls = omegas_ls-np.mean(np.diff(omegas_ls))/2
QQ,WW_ls = np.meshgrid(Q,omegas_ls)

blur_ls = 1
gf_ls = lambda x: gaussian_filter1d(x,blur_ls,axis=0)
ls_filt = gf_ls(ls)
plt.figure()
plt.pcolormesh(Q,omegas_ls,np.abs(WW_ls)*ls_filt)
# plt.ylim([-90,90])
plt.xlim([0.5,4])
plt.clim([0,1e-4])
plt.colorbar()
plt.xlabel(r'Q ($\AA^{-1}$)')
plt.ylabel(r'$\omega$ (rad THz)')
plt.title('Lomb-Scargle Data (20 fs timebins)')
plt.show()

# Theory comparison

In [ ]:
theory = {}

import h5py
with h5py.File('cs2_diffsig_theory.mat','r') as f:
    for name, data in f.items():
        theory[name]=f[name].value

# qAng - Dimensions (1, Nq) - q in inverse angstroms. 
# tt - Dimensions (1, nts) - Time vector.
# tc - Dimensions (1,ntc) - Extended time vector for convoluted signal.
# Wiam - Dimensions (Ntraj, Ntraj, Nq, nts) - Full scattering matrix (Imol + Iat), diagonal elements only.
# Iat - Dimensions (1, Nq) - Atomic scattering term. 
# WW - Dimensions (Nq, nts) - Total scattering for Ehrenfest wavefunction (just the average over Ntraj as equal weights) - all trajectories. 
# WWc - Dimensions (Nq, nts) - Convoluted signal - all trajectories.
# dW - Dimensions (Nq, nts) - Unconvoluted (raw) percentage difference signal - all trajectories.
# dWc - Dimensions (Nq, nts) - Convoluted percentage difference signal - all trajectories. 

In [ ]:
def rebin_theory(ts, ts_rebin, signal):
    dt = np.diff(ts_rebin)
    output = np.zeros((ts_rebin.size,Qs.size))
    for i,t in enumerate(ts_rebin[:-1]):
        tIdx = (ts>t)&(ts<t+dt[i])
        output[i,:] = np.nanmean(signal[tIdx,:],axis=0)
    return output

Qs, ts = theory['qAng'][(theory['qAng']>0.5)&(theory['qAng']<8)].flatten(), theory['tc'].flatten()
dWc = theory['dWc'][:,(theory['qAng']>0.5).flatten()&(theory['qAng']<8).flatten()]
print Qs.shape, ts.shape, dWc.shape
plt.figure()
plt.pcolormesh(Qs,ts,dWc)
plt.colorbar()
plt.xlim([0.5,8])
plt.show()

tbin_size = 2
ts_rebin = np.arange(np.min(ts),np.max(ts),tbin_size)
signal_rebin = rebin_theory(ts,ts_rebin,dWc)
# signal_rebin = gf_q(signal_rebin)
# signal_rebin = gf_t(signal_rebin)
plt.figure()
plt.pcolormesh(Qs,ts_rebin,signal_rebin)
plt.colorbar()
plt.xlim([0.5,8])
plt.show()

In [ ]:
tmin,tmax = 200,1200
ts = ts_rebin
dWc = signal_rebin

ts_transform = ts[(ts>tmin)&(ts<tmax)]
dWc_transform = dWc[(ts>tmin)&(ts<tmax),:]
dWc_norm = dWc - 1*np.mean(dWc_transform,axis=0)
omegas = np.fft.fftshift(np.fft.fftfreq(n=ts_transform.size,d=np.mean(np.diff(ts_transform)/1000.)))*2*np.pi
dw = np.mean(np.diff(omegas))
omegas = omegas-dw/2
QQ,WW = np.meshgrid(Qs,omegas)
dWc_norm = dWc_transform - np.mean(dWc_transform,axis=0)
fft_theory = np.fft.fftshift(np.fft.fft(dWc_norm,axis=0),axes=0)
# fft_theory = gf_f(fft_theory)
plt.figure()
plt.pcolormesh(Qs,omegas,np.abs(WW)*np.abs(fft_theory))
plt.colorbar()
plt.xlim([0.5,8])
plt.ylim([-np.min((np.max(omegas),400)),np.min((np.max(omegas),400))])
plt.clim([0,1000])
plt.xlabel(r'Q ($\AA^{-1}$)')
plt.ylabel(r'$\omega$ (rad THz)')
plt.title('CS2 FRXS Theory (%d fs timebins)' % tbin_size)
plt.show()

In [ ]:
omegas_ls_theory, ls_theory = get_lombscargle(Qs,ts_transform,dWc_norm)
# ls_theory = gf_ls(ls_theory)
QQ_ls_theory,WW_ls_theory = np.meshgrid(Qs,omegas_ls_theory)
plt.figure()
plt.pcolormesh(Qs,omegas_ls_theory,np.abs(WW_ls_theory)*ls_theory)
plt.ylim([-np.min((np.max(omegas_ls_theory),400)),np.min((np.max(omegas_ls_theory),400))])
plt.xlim([0.5,8])
plt.clim([0,10])
plt.colorbar()
plt.xlabel(r'Q ($\AA^{-1}$)')
plt.ylabel(r'$\omega$ (rad THz)')
plt.title('CS2 Lomb-Scargle Theory (%d fs timebins)' % tbin_size)
plt.show()

## Check that Lomb-Scargle gives nice results for simple trajectories

In [ ]:
ts = np.linspace(0,1000,200)
qs = np.linspace(1,4,100)
f = 200
a = 0.1
def r(t,f,a):
    return a*np.sin(f*t/1000)+2.5
def r2(t,v,a):
    return 2.5+v*t/1000
QQ,TT = np.meshgrid(qs,ts)
s1 = np.sinc(QQ*r2(TT,f/2,a)/np.pi)
s2 = np.sinc(QQ*r(TT,1000,0.1))
s = s1+s2
plt.figure()
plt.plot(ts,r2(ts,f,a))
plt.title('Trajectory')
plt.show()

plt.figure()
plt.pcolormesh(QQ,TT,s)
plt.colorbar()
plt.title('S(Q,t)')
plt.show()
print s.shape,QQ.shape,TT.shape

wws,ls_ss = get_lombscargle(qs,ts,s)

plt.figure()
plt.pcolormesh(qs,wws,ls_ss)
plt.colorbar()
plt.clim([0,np.max(ls_ss)/10])
plt.title('Lomb-Scargle')
plt.show()

omegas = np.fft.fftshift(np.fft.fftfreq(len(ts), d=np.mean(np.diff(ts))),axes=0)*1e3*2*np.pi
ft = np.fft.fftshift(np.fft.fft(s-np.mean(s,axis=0),axis=0),axes=0)

plt.figure()
plt.pcolormesh(qs,omegas,np.abs(ft))
plt.colorbar()
plt.title('FFT')
plt.clim([0,1])
plt.show()